<a href="https://colab.research.google.com/github/ampehta/color-bert/blob/main/TrainTpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Configuration

In [1]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--max_len',default=256)
parser.add_argument('--batch_size_TPU',default=32)
parser.add_argument('--batch_size',default=64)
parser.add_argument('--epochs',default=10)
parser.add_argument('--learning_rate',default=1e-5)
parser.add_argument('--evaluate_every',default=5)
parser.add_argument('--vanilla_bert',default='bert-base-uncased')

args=parser.parse_args(args=[])

## Connect TPU

In [2]:
#from model.py
import tensorflow as tf

def connect_TPU(BatchSizeTpu):
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    global_batch_size = BatchSizeTpu * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size

In [3]:
#from network.py
TPU, Strategy, GlobalBatchSize = connect_TPU(args.batch_size_TPU)

INFO:tensorflow:Initializing the TPU system: grpc://10.25.219.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.25.219.186:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


##Get Sample Dataset & Tokenize

In [4]:
import urllib

url = "https://raw.githubusercontent.com/muhwagua/color-bert/no-raw/data/masked.txt"
file = urllib.request.urlopen(url)
mask = []
for line in file:
	decoded_line = line.decode("utf-8")
	mask.append(decoded_line.strip())

url = "https://raw.githubusercontent.com/muhwagua/color-bert/no-raw/data/labels.txt"
file = urllib.request.urlopen(url)
label = []
for line in file:
	decoded_line = line.decode("utf-8")
	label.append(decoded_line.strip())
 
print(len(mask),len(label))

9760 9760


In [5]:
!pip install transformers

     |████████████████████████████████| 2.0MB 6.2MB/s 
     |████████████████████████████████| 890kB 38.6MB/s 
     |████████████████████████████████| 3.2MB 36.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=8a31f70eac3fe6b0a60b920fc69f58d0c8b80c4a4b20ea52629d6ff427ebc9f1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

train_text = tokenizer.batch_encode_plus(label[:6000],return_attention_mask=False,return_token_type_ids=False,padding=True,truncation=True,max_length=256)['input_ids']
train_label = tokenizer.batch_encode_plus(mask[:6000],return_attention_mask=False,return_token_type_ids=False,padding=True,truncation=True, max_length=256)['input_ids']

print(len(train_text),len(train_label))


6000 6000


##Distribute Dataset

In [7]:
#from model.py
def create_distributed_dataset(Strategy,X, y=None, global_batch_size = 16,training=False):
    dataset = tf.data.Dataset.from_tensor_slices(X)
    AUTO = tf.data.experimental.AUTOTUNE
    if y is not None:
        dataset_y = tf.data.Dataset.from_tensor_slices(y)
        dataset = tf.data.Dataset.zip((dataset, dataset_y))

    if training:
        dataset = dataset.shuffle(len(X)).repeat()

    dataset = dataset.batch(global_batch_size).prefetch(AUTO)
    dist_dataset = Strategy.experimental_distribute_dataset(dataset)

    return dist_dataset

In [8]:
#from network.py
dist_dataset = create_distributed_dataset(Strategy,train_text,train_label,GlobalBatchSize)

## Get Model&Optimizer
#### For Saved Models need Modification

In [9]:
#from model.py

def get_model_and_optimizer(Strategy,model,LR,checkpoint=False):
    with Strategy.scope():
        if checkpoint:
            pass
        else:
            model = TFBertForMaskedLM.from_pretrained(model,output_attentions=True)
            optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    return model, optimizer

In [10]:
#from network.py
from transformers import TFBertForMaskedLM
Model , Optimizer = get_model_and_optimizer(Strategy,args.vanilla_bert,args.learning_rate)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


## Train Model

In [11]:
#from model.py
from datetime import datetime

class TrainTpu(tf.keras.Model):
    def __init__(self, dataset, model, optimizer, args, Tpu, Strategy, GlobalBatchSize):
        super(TrainTpu, self).__init__()
        self.dataset = dataset
        self.Strategy = Strategy
        self.MLM_Bert = model
        self.GlobalBatchSize = GlobalBatchSize
        self.Epoch = args.epochs
        self.evaluate_every = args.evaluate_every
        self.Optimizer = optimizer
        self.checkpoint_path = 'MLM_BERT/training/Checkpoint'

    def __call__(self):  # Train Distributed MLM
        step = 0
        self.MLM_loss_func, self.MLM_train_loss = self.define_mlm_loss_and_metrics()

        for step in range(self.Epoch):
            for tensor in self.dataset:
                self.distributed_MLM_train_step(tensor)
            step += 1
            print(step)
            if step % self.evaluate_every == 0:
                train_metric = self.MLM_train_loss.result().numpy()
                print("Step %d, train loss: %.2f" % (step, train_metric))
                self.MLM_train_loss.reset_states()
                #self.MLM_Bert.save("Color_Bert/ex")

           # if step % 10==0: # For Checkpoints
           #     checkpoint = tf.train.Checkpoint(model=self.MLM_Bert)
           #     local_device_option = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")

           #     checkpoint_path = "MLM_BERT/cp-{step}.ckpt"
           #     checkpoint_dir = os.path.dirname(checkpoint_path)
           #     checkpoint.write(checkpoint_path, options=local_device_option)


    @tf.function
    def distributed_MLM_train_step(self, data):
        self.Strategy.run(self.MLM_train_step, args=(data,))

    @tf.function
    def MLM_train_step(self, data):
        Feature, Label = data  # dataloader의 인풋에 맞추어 바꿔야함

        with tf.GradientTape() as tape:
            prediction = self.MLM_Bert(Feature, training=True)[0]
            loss = self.MLM_loss_func(Label, prediction)

        gradients = tape.gradient(loss, self.MLM_Bert.trainable_variables)
        self.Optimizer.apply_gradients(zip(gradients, self.MLM_Bert.trainable_variables))
        self.MLM_train_loss.update_state(loss)

    def define_mlm_loss_and_metrics(self):
        with self.Strategy.scope():
            mlm_loss_object = self.masked_sparse_categorical_crossentropy

            def MLM_loss(labels, predictions):
                per_example_loss = mlm_loss_object(labels, predictions)
                loss = tf.nn.compute_average_loss(
                    per_example_loss, global_batch_size=self.GlobalBatchSize)
                return loss

            train_mlm_loss_metric = tf.keras.metrics.Mean()

        return MLM_loss, train_mlm_loss_metric

    def masked_sparse_categorical_crossentropy(self, y_true, y_pred):
        y_true_masked = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
        y_pred_masked = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))

        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_masked,
                                                               y_pred_masked,
                                                               from_logits=True)
        return loss

In [12]:
#from network.py
train = TrainTpu(dist_dataset,Model,Optimizer,args,TPU,Strategy,GlobalBatchSize) # train_dist_dataset : dataset, model: bert-based-uncased

In [13]:
%%time
train()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


1
2
3
4
5
Step 5, train loss: 66.86
6
7
8
9
10
Step 10, train loss: 15.53
CPU times: user 42.8 s, sys: 2.85 s, total: 45.7 s
Wall time: 5min 23s
